# Burgers Equation Identification
Equation:   $u_{t} + \lambda_1 uu_{x}-\lambda_2 u_{xx} = 0$  

In [67]:
#@uthor : $um@nth107

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, InputLayer, Lambda, Layer, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_normal

##Data Prepocessing

In [73]:
N_u = 2000
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat') 
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  
u_star = Exact.flatten()[:,None]                                                 
lb = X_star.min(0)                                                              
ub = X_star.max(0)
np.random.seed(107)
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_train = tf.convert_to_tensor(X_star[idx,:], dtype='float64')
u_train = tf.convert_to_tensor(u_star[idx,:], dtype='float64')
x = tf.convert_to_tensor(X_train[:,0:1], dtype='float64')         
t = tf.convert_to_tensor(X_train[:,1:2], dtype='float64')

##PINN

In [74]:
input = Input(shape=(layers[0],))
input_mod = Lambda(lambda X: 2.0*(X - lb)/(ub - lb) - 1.0)(input)
dummy_input = Input(shape=(layers[0],))
dummy_output = Dense(units=1, use_bias=False)(dummy_input)
flag = 0
for i in layers[1:-1]:
  if flag == 0:
    output = Dense(units=i, activation="tanh", kernel_initializer="glorot_normal")(input_mod) 
    flag += 1
  else:
    output = Dense(units=i, activation="tanh", kernel_initializer="glorot_normal")(output)
output = Dense(units=layers[-1], kernel_initializer="glorot_normal")(output)
pinn_model = Model(inputs = [input, dummy_input], outputs = [output, dummy_output])

def custom_loss(model, dummy_layer, x, t):
  def loss(u_train, u_pred):
    lambda1 = tf.cast(dummy_layer.trainable_weights[0][0], dtype='float64')
    lambda2 = tf.cast(dummy_layer.trainable_weights[0][1], dtype='float64')
    with tf.GradientTape(persistent=True) as tape:
      tape.watch(x)
      tape.watch(t)
      xt = tf.stack([x[:,0], t[:,0]], axis=1) 
      u = tf.cast(model(([xt, np.random.normal(size=(2000, 2))]))[0], dtype='float64')
      u_x = tape.gradient(u, x)
    u_xx = tape.gradient(u_x, x)
    u_t = tape.gradient(u, t)
    del tape
    f = u_t + lambda1*u*u_x - lambda2*u_xx
    loss1 = tf.cast(tf.reduce_mean(tf.square(u_pred - u_train)), dtype='float64')
    loss2 = tf.reduce_mean(tf.square(f))
    return loss1 + loss2
  return loss

def dummy_loss(u_train, u_pred):
  return 0.0

loss_fn = custom_loss(pinn_model, pinn_model.layers[-1], x, t)
pinn_model.compile(optimizer=Adam(learning_rate=0.0005), loss=[loss_fn, dummy_loss], loss_weights=[1.0, 0.0])
pinn_model.fit([X_train, np.random.normal(size=(N_u, 2))], [u_train, u_train], batch_size=128, epochs=2500, verbose=1)

Streaming output truncated to the last 5000 lines.
Epoch 1/2500
16/16 [==============================] - 2s 22ms/step - loss: 0.4088 - dense_266_loss: 0.4088 - dense_257_loss: 0.0000e+00
Epoch 2/2500
16/16 [==============================] - 0s 22ms/step - loss: 0.2960 - dense_266_loss: 0.2960 - dense_257_loss: 0.0000e+00
Epoch 3/2500
16/16 [==============================] - 0s 22ms/step - loss: 0.2764 - dense_266_loss: 0.2764 - dense_257_loss: 0.0000e+00
Epoch 4/2500
16/16 [==============================] - 0s 23ms/step - loss: 0.2665 - dense_266_loss: 0.2665 - dense_257_loss: 0.0000e+00
Epoch 5/2500
16/16 [==============================] - 0s 22ms/step - loss: 0.2688 - dense_266_loss: 0.2688 - dense_257_loss: 0.0000e+00
Epoch 6/2500
16/16 [==============================] - 0s 21ms/step - loss: 0.2803 - dense_266_loss: 0.2803 - dense_257_loss: 0.0000e+00
Epoch 7/2500
16/16 [==============================] - 0s 22ms/step - loss: 0.2647 - dense_266_loss: 0.2647 - dense_257_loss: 0.0000e+

##Predicted $\lambda_1$ & $\lambda_2$ values

In [82]:
lambda1 = float(pinn_model.layers[-1].trainable_weights[0][0])
lambda2 = float(pinn_model.layers[-1].trainable_weights[0][1])
print('Lambda1 Pred:', round(lambda1,8), '  ; Lambda1 Actual:', 1.0)
print('Lambda2 Pred:', round(lambda2,8), '; Lambda2 Actual:', round(0.01/np.pi,8))

Lambda1 Pred: 0.986283   ; Lambda1 Actual: 1.0
Lambda2 Pred: 0.00399964 ; Lambda2 Actual: 0.0031831
